In [1]:
from qiskit.opflow import I, X, Y, Z, StateFn, CircuitSampler
from qiskit.circuit import QuantumCircuit
from qiskit.opflow import Z, X
from qiskit.opflow import CircuitSampler, PauliExpectation, ExpectationFactory,AerPauliExpectation,MatrixExpectation
import time
from qiskit.circuit import Parameter, ParameterVector
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from qiskit import Aer
from qiskit.utils import QuantumInstance

backend = Aer.get_backend('qasm_simulator')
q_instance = QuantumInstance(backend, shots = 8192, seed_simulator = 2718, seed_transpiler = 2718)

In [3]:
# Define quantum circuit
H = X ^ X

U = QuantumCircuit(2)
U.h(0)
U.cx(0, 1)

Takeaways:
- For a simple hamiltonian, .eval is faster due to the overhead of the conversions for PauliExpectation and CircuitSampler. - For a more complex hamiltonian, the use of PauliExpectation and CircuitSampler speeds up the process.
- ExpectationFactory returns the most efficient Expectation converter choice ("one that best suits the operator and    backend") - we can use this to cater to our specific circuit!

In [19]:
# Compute expectation value using .eval()
expectation = StateFn(H, is_measurement=True) @ StateFn(U)
# Time .eval execution
start = time.time()
matmult_result = expectation.eval()
end = time.time()
print(matmult_result)
print("Time elapsed: ", end - start)

(0.9999999999999998+0j)
Time elapsed:  0.005015134811401367


In [5]:
# Compute expectation value using circuit sampler and Pauli expectation
sampler = CircuitSampler(q_instance)

in_pauli_basis = PauliExpectation().convert(expectation)
# Time 
start = time.time()
shots_result = sampler.convert(in_pauli_basis).eval()
end = time.time()
print(shots_result)
print("Time elapsed: ", end - start)

(1+0j)
Time elapsed:  0.07185482978820801


Expectation Testing in isolation
- Matrix and Pauli similar
- Not totally sure how AerPauliExpectation works

In [18]:
# Compute expectation value using Pauli expectation & no sampler
# Note that for Pauli expectation, we need to use .eval to evaluate vs 
# for aer pauli expectation we just use .convert

in_pauli_basis = PauliExpectation().convert(expectation)
# Time
start = time.time()
shots_result = (in_pauli_basis).eval()
end = time.time()
print(shots_result)
print("Time elapsed: ", end - start)

(0.9999999999999997+0j)
Time elapsed:  0.007700920104980469


In [7]:
# Compute expectation value using matrix expectation & no sampler
# Note that for Pauli expectation, we need to use .eval to evaluate vs 
# for aer pauli expectation we just use .convert

matrix_ex = MatrixExpectation().convert(expectation)
# Time
start = time.time()
shots_result = (matrix_ex).eval()
end = time.time()
print(shots_result)
print("Time elapsed: ", end - start)

(0.9999999999999998+0j)
Time elapsed:  0.005978584289550781


In [8]:
# Compute expectation value using recommendation of expectation factory & no sampler
expectation_factory = ExpectationFactory().build(H)
print(expectation_factory)

# Time
start = time.time()
in_aer_pauli_basis = AerPauliExpectation().convert(expectation)
#shots_result = (in_aer_pauli_basis).eval()
end = time.time()
print(in_aer_pauli_basis)
print("Time elapsed: ", end - start)

ComposedOp([
  CircuitMeasurement(
        ░ 
  q_0: ─░─
        ░ 
  q_1: ─░─
        ░ 
  ),
  CircuitStateFn(
       ┌───┐     
  q_0: ┤ H ├──■──
       └───┘┌─┴─┐
  q_1: ─────┤ X ├
            └───┘
  )
])
Time elapsed:  0.0011432170867919922


Now let's try with a slightly more complicated Hamiltonian!
- Here, matrix expectation is fastest followed by .eval, and pauli expectation + circuit sampler is significantly slower

In [9]:
hamiltonian = (Z ^ X) +  (X ^ Z)
U2 = QuantumCircuit(4)
U2.h(0)
U2.cx(0, 1)
U2.cx(1,2)
U2.cx(1,3)
U2.rz(2*np.pi,2)

In [20]:
h2_measurement = StateFn(hamiltonian, is_measurement=True) @ StateFn(U2)
print(h2_measurement)

ComposedOp([
  OperatorMeasurement(1.0 * ZXII
  + 1.0 * XZII),
  CircuitStateFn(
       ┌───┐                         
  q_0: ┤ H ├──■──────────────────────
       └───┘┌─┴─┐                    
  q_1: ─────┤ X ├──■────■────────────
            └───┘┌─┴─┐  │  ┌────────┐
  q_2: ──────────┤ X ├──┼──┤ Rz(2π) ├
                 └───┘┌─┴─┐└────────┘
  q_3: ───────────────┤ X ├──────────
                      └───┘          
  )
])


In [22]:
# Using Pauli Expectation and CircuitSampler

sampler = CircuitSampler(q_instance)

in_pauli_basis = PauliExpectation().convert(h2_measurement)
# Time
start = time.time()
shots_result = sampler.convert(in_pauli_basis).eval()
end = time.time()
print(shots_result)
print("Time elapsed: ", end - start)

(0.012207031250000028+0j)
Time elapsed:  0.26180481910705566


In [23]:
# Using eval
start = time.time()
matmult_result = h2_measurement.eval()
end = time.time()
print(matmult_result)
print("Time elapsed: ", end - start)

0j
Time elapsed:  0.007428169250488281


In [24]:
# Using Pauli Expectation and no sampler
start = time.time()
shots_result = PauliExpectation().convert(h2_measurement).eval()
end = time.time()
print(shots_result)
print("Time elapsed: ", end - start)

0j
Time elapsed:  0.040735483169555664


In [25]:
# Using matrix expectation and no sampler
matrix_ex = MatrixExpectation().convert(h2_measurement)
# Time
start = time.time()
shots_result = (matrix_ex).eval()
end = time.time()
print(shots_result)
print("Time elapsed: ", end - start)

0j
Time elapsed:  0.006113529205322266


In [26]:
# Using expectation factory recommendation
expectation_factory = ExpectationFactory().build(hamiltonian)
print(expectation_factory)

# Time
start = time.time()
in_aer_pauli_basis = AerPauliExpectation().convert(h2_measurement)
end = time.time()
print(in_aer_pauli_basis)
print("Time elapsed: ", end - start)

ComposedOp([
  CircuitMeasurement(
        ░ 
  q_0: ─░─
        ░ 
  q_1: ─░─
        ░ 
  q_2: ─░─
        ░ 
  q_3: ─░─
        ░ 
  ),
  CircuitStateFn(
       ┌───┐                         
  q_0: ┤ H ├──■──────────────────────
       └───┘┌─┴─┐                    
  q_1: ─────┤ X ├──■────■────────────
            └───┘┌─┴─┐  │  ┌────────┐
  q_2: ──────────┤ X ├──┼──┤ Rz(2π) ├
                 └───┘┌─┴─┐└────────┘
  q_3: ───────────────┤ X ├──────────
                      └───┘          
  )
])
Time elapsed:  0.0010809898376464844
